In [1]:
import pandas as pd
from glob import glob
import networkx as nx
import numpy as np

In [2]:
files =  glob('../../../../../../../../Volumes/sandisk1/datasets/splicing/Athma_cryptic_splicing/splicesites_for_cshl/*.csv')
dfs = []
for f in files:
    dfs.append(pd.read_csv(f,dtype={"chr": str, "site": int, 'strand':str, 'splicesite':str,'STARannot':int,'dinuc':str,'cell':str,
                                    'rna_type':str,
                                    'reads':int,
                                    'gene':str,
                                    'cryptic_status':str,
                                    'cryptic_type':str,
                                    'exon':str,
                                    'constitutiveexon':str,
                                    'intron':str,
                                    'constitutiveintron':str,
                                    'tpm':float}))
dfs = pd.concat(dfs).reset_index()
dfs

,index,chr,site,strand,splicesite,STARannot,dinuc,cell,rna_type,reads,gene,cryptic_status,cryptic_type,exon,constitutiveexon,intron,constitutiveintron,tpm
0,0,10,100001026,-,3ss,0,GA,KNS60,nascent,5,ENSG00000107554,cryptic,LowFidelity,0,NaN,ENSG00000107554:99972135-100009837,0,42.279118
1,1,10,100001644,-,5ss,0,GC,KNS60,nascent,5,ENSG00000107554,cryptic,HighFidelity,0,NaN,ENSG00000107554:99972135-100009837,0,42.279118
2,2,10,100002941,-,5ss,0,GT,KNS60,nascent,30,ENSG00000107554,cryptic,HighFidelity,0,NaN,ENSG00000107554:99972135-100009837,0,42.279118
3,3,10,100003024,-,3ss,0,AG,KNS60,nascent,30,ENSG00000107554,cryptic,HighFidelity,0,NaN,ENSG00000107554:99972135-100009837,0,42.279118
4,4,10,100006236,-,5ss,0,GT,KNS60,nascent,7,ENSG00000107554,cryptic,recursive,0,NaN,ENSG00000107554:99972135-100009837,0,42.279118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2622128,364130,X,9939367,+,5ss,1,GT,KNS60,polya,94,ENSG00000146950,canonical,canonical,0,NaN,ENSG00000146950:9939367-9944640;ENSG0000014695...,0,3.752241
2622129,364131,X,9944640,+,3ss,1,AG,KNS60,polya,75,ENSG00000146950,canonical,canonical,0,NaN,ENSG00000146950:9939367-9944640;ENSG0000014695...,0,3.752241
2622130,364132,X,9944643,+,3ss,1,AG,KNS60,polya,19,ENSG00000146950,canonical,canonical,ENSG00000146950:9944641-9944913,0,ENSG00000146950:9939367-9944643,0,3.752241
2622131,364133,X,9944914,+,5ss,1,GT,KNS60,polya,114,ENSG00000146950,canonical,canonical,0,NaN,ENSG00000146950:9944914-9946670,0,3.752241


In [3]:
full_tpm_df = pd.read_csv('../../../../../../../../Volumes/sandisk1/datasets/splicing/Athma_cryptic_splicing/tpms_K562_KNS60_112924.csv', index_col=0)
full_tpm_df.columns = pd.MultiIndex.from_tuples([(r1, r2) for r1, r2 in full_tpm_df.columns.str.split('_')], names=['cell','rna_type'])
full_tpm_df = full_tpm_df.loc[dfs.gene.unique()]
full_tpm_df = full_tpm_df.stack(['cell', 'rna_type']).rename('tpm').to_frame().sort_index()

full_tpm_df.to_csv('../../../../../../../../Volumes/sandisk1/datasets/splicing/Athma_cryptic_splicing/processed_round1/tpms.csv')
full_tpm_df

/var/folders/5_/tjkj20xs1zjbm1r6n07ccp4h0000gn/T/ipykernel_56013/2407871938.py:4: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  full_tpm_df = full_tpm_df.stack(['cell', 'rna_type']).rename('tpm').to_frame().sort_index()


tpm
genes           cell  rna_type                
ENSG00000000003 K562  cycloheximide   0.054959
                      nascent         0.007408
                      nuclear         0.110831
                      polya           0.038089
                KNS60 cycloheximide  10.946019
...                                        ...
ENSG00000283696 K562  polya           1.582833
                KNS60 cycloheximide   0.616325
                      nascent         4.077657
                      nuclear         1.198403
                      polya           1.140936

[120008 rows x 1 columns]

In [4]:
junction_df = dfs.set_index(['gene', 'chr', 'strand', 'site'])[['splicesite','cell', 'rna_type', 'dinuc','cryptic_type','reads']].sort_index()
junction_df.to_csv('../../../../../../../../Volumes/sandisk1/datasets/splicing/Athma_cryptic_splicing/processed_round1/junctions.csv')
junction_df

splicesite   cell       rna_type dinuc  \
gene            chr strand site                                               
ENSG00000000003 X   -      100629987        3ss  KNS60        nascent    AG   
                           100629987        3ss  KNS60  cycloheximide    AG   
                           100629987        3ss  KNS60        nuclear    AG   
                           100629987        3ss  KNS60          polya    AG   
                           100630758        5ss  KNS60        nascent    GT   
...                                         ...    ...            ...   ...   
ENSG00000283689 16  +      54288560         3ss  KNS60  cycloheximide    AG   
                           54288560         3ss  KNS60        nuclear    AG   
                           54288560         3ss  KNS60          polya    AG   
ENSG00000283696 1   +      161399913        5ss   K562        nascent    GT   
                           161421931        3ss   K562        nascent    AG   

                                     cryptic_type  reads  
gene            chr strand site                           
ENSG00000000003 X   -      100629987    canonical     41  
                           100629987    canonical    302  
                           100629987    canonical    691  
                           100629987    canonical    979  
                           100630758    canonical     41  
...                                           ...    ...  
ENSG00000283689 16  +      54288560     canonical     64  
                           54288560     canonical     30  
                           54288560     canonical    108  
ENSG00000283696 1   +      161399913    canonical     15  
                           161421931    canonical     15  

[2622133 rows x 6 columns]

In [5]:
para = pd.read_csv('../../../../../../../../Volumes/sandisk1/datasets/splicing/Athma_cryptic_splicing/GRCh38_ensembl95.Paralogs.csv')
para


,Unnamed: 0,Gene Internal ID,Gene stable ID,Gene Synbol,HasParalog,Paralog stable ID,Paralog Synbol
0,0,319832,ENSG00000284532,MIR4723,False,NaN,NaN
1,1,319833,ENSG00000238933,RF00019,True,ENSG00000202361,RF00019
2,2,319833,ENSG00000238933,RF00019,True,ENSG00000201843,RF00019
3,3,319833,ENSG00000238933,RF00019,True,ENSG00000200008,RF00019
4,4,319833,ENSG00000238933,RF00019,True,ENSG00000201466,RF00019
...,...,...,...,...,...,...,...
3618911,3618911,324201,ENSG00000285013,AC243837.3,False,NaN,NaN
3618912,3618912,324202,ENSG00000285154,AC243837.4,False,NaN,NaN
3618913,3618913,324203,ENSG00000284933,ARHGEF5,False,NaN,NaN
3618914,3618914,324204,ENSG00000285363,MTRF1LP2,False,NaN,NaN


In [6]:
gene_data = pd.read_csv('../../../../../../../../Volumes/sandisk1/datasets/splicing/Athma_cryptic_splicing/GRCh38_ensembl95.genes.csv')
gene_data = gene_data.rename(columns={'Gene Chromosome':'Chromosome/scaffold name', 'Gene Strand':'Strand','Gene Synbol':'Gene name'})
gene_data = gene_data[['Gene stable ID','Chromosome/scaffold name','Gene start (bp)','Gene end (bp)','Strand','Gene name']].set_index('Gene stable ID')
print(gene_data.index.has_duplicates)
gene_data.to_csv('../../../../../../../../Volumes/sandisk1/datasets/splicing/Athma_cryptic_splicing/processed_round1/gene_data.csv')
gene_data

False


,Chromosome/scaffold name,Gene start (bp),Gene end (bp),Strand,Gene name
Gene stable ID,,,,,
ENSG00000284532,17,28360654,28360734,1,MIR4723
ENSG00000238933,9,61642383,61642494,1,RF00019
ENSG00000275693,10,88992370,88992539,1,RF02116
ENSG00000275451,15,62343029,62343138,1,MIR6085
ENSG00000222870,7,94495299,94495404,1,RNU6-1328P
...,...,...,...,...,...
ENSG00000285013,CHR_HSCHR7_3_CTG4_4,144396023,144396316,1,AC243837.3
ENSG00000285154,CHR_HSCHR7_3_CTG4_4,144414782,144415702,1,AC243837.4
ENSG00000284933,CHR_HSCHR7_3_CTG4_4,144457568,144474951,1,ARHGEF5


In [7]:
only_observed_orthos = para[para['Gene stable ID'].isin(dfs['gene'].unique())&para['Paralog stable ID'].isin(dfs['gene'].unique())]
only_observed_orthos = only_observed_orthos[['Gene stable ID', 'Paralog stable ID']]

only_observed_orthos = only_observed_orthos.apply(lambda x: (x['Gene stable ID'],x['Paralog stable ID']),
                                                            axis=1)
only_observed_orthos

219472     (ENSG00000198886, ENSG00000198786)
219575     (ENSG00000198786, ENSG00000198886)
356712     (ENSG00000142552, ENSG00000128595)
356713     (ENSG00000142552, ENSG00000117906)
356714     (ENSG00000142552, ENSG00000078808)
                          ...                
2390973    (ENSG00000120451, ENSG00000109762)
2390974    (ENSG00000120451, ENSG00000135317)
2391089    (ENSG00000187735, ENSG00000204219)
2391090    (ENSG00000187735, ENSG00000171703)
2391091    (ENSG00000187735, ENSG00000176896)
Length: 90528, dtype: object

In [8]:
G =  nx.Graph()
G.add_nodes_from(dfs.groupby('gene').first().apply(lambda x: (x.name, {'chr':x['chr'], 'strand':x['strand']}), axis=1))
G.add_edges_from(only_observed_orthos)
print(f'{G.number_of_nodes()=}')

all_nodes = pd.Series(G.nodes).index
print('These nodes are missing from the paralog search')
print(all_nodes[~all_nodes.isin(para['Gene stable ID'].unique())])

ortho_groups = list(nx.connected_components(G))
print(f'captured {len(ortho_groups)} ortho groups')

test_set = [
    (G.degree[node], node)
    for node, data in G.nodes(data=True) 
    if data.get('chr') in ['1', '3', '5', '7', '9'] and G.degree[node] == 0
]

neghbors, test_nodes = zip(*test_set)
print(np.unique(neghbors))
print(len(test_nodes))

G.remove_nodes_from(test_nodes)


ortho_groups = list(nx.connected_components(G))
len(ortho_groups)

rng = np.random.default_rng(42)

perm = rng.permutation(len(ortho_groups))
chunks = np.array_split(perm, 5)
cvs = [[ortho_groups[i] for i in chunk] for chunk in chunks ]
cvs = [set.union(*cv) for cv in cvs]


G.number_of_nodes()=15001
These nodes are missing from the paralog search
Index([], dtype='object')
captured 8076 ortho groups
[0]
1767


In [9]:
gene_to_split = pd.DataFrame()
gene_to_split['gene'] = list(set.union(set(test_nodes), set.union(*cvs)))

gene_to_split['Test_set'] = False
gene_to_split.loc[gene_to_split['gene'].isin(test_nodes),'Test_set'] = True

for i, cv in enumerate(cvs):
    gene_to_split[f'CV{i}'] = False
    gene_to_split.loc[gene_to_split['gene'].isin(cv),f'CV{i}'] = True

gene_to_split.to_csv('../../../../../../../../Volumes/sandisk1/datasets/splicing/Athma_cryptic_splicing/processed_round1/gene_split.csv')
gene_to_split

,gene,Test_set,CV0,CV1,CV2,CV3,CV4
0,ENSG00000091073,False,False,False,False,True,False
1,ENSG00000120616,False,True,False,False,False,False
2,ENSG00000158796,False,False,True,False,False,False
3,ENSG00000233006,True,False,False,False,False,False
4,ENSG00000175040,False,False,True,False,False,False
...,...,...,...,...,...,...,...
14996,ENSG00000132912,True,False,False,False,False,False
14997,ENSG00000025039,False,False,False,False,True,False
14998,ENSG00000259768,False,False,False,False,True,False
14999,ENSG00000114098,True,False,False,False,False,False


In [10]:
gene_to_split.drop(columns='gene').sum()

Test_set    1767
CV0         2502
CV1         2756
CV2         2758
CV3         2518
CV4         2700
dtype: int64